# Granite Dells

https://www.google.com/maps/dir//34.5936281,-112.4193333/@34.5933671,-112.4193692,256m/data=!3m1!1e3!4m2!4m1!3e0?entry=ttu&g_ep=EgoyMDI0MTIwNC4wIKXMDSoASAFQAw%3D%3D

## 1. Organize data

Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

Using SfM products from Agisoft:
```
semantic_SfM/data
    ├── granite_dells
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── cameras.xml
        │       ├── granite_dells_wgs_utm.jpg
        │       ├── granite_dells_wgs_utm.obj
        │       ├── granite_dells_wgs_utm.mtl
        │       └── granite_dells_wgs_utm.las   
        ├── segmentations
        └── associations

```

In [2]:
import os

scene_dir = '../../data/granite_dells'
pointcloud_path = os.path.join(scene_dir, 'SfM_products', 'granite_dells_wgs_utm.las')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
photos_folder_path = os.path.join(scene_dir, 'DJI_photos')
camera_path = os.path.join(scene_dir, 'SfM_products', 'cameras.xml')
mesh_path = os.path.join(scene_dir, 'SfM_products', 'granite_dells_wgs_utm.obj')

In [3]:
downsampling= False

if downsampling == True:
    from ssfm.downsample_pointcloud import downsample_pointcloud
    downsampled_pointcloud_path = os.path.join(scene_dir, 'SfM_products', 'granite_dells_downsampled.las')
    downsample_pointcloud(pointcloud_path, downsampled_pointcloud_path, "uniform", 4)


downsampled_pointcloud_path = os.path.join(scene_dir, 'SfM_products', 'granite_dells_downsampled.las')
pointcloud_path = downsampled_pointcloud_path

## 2. Create 2D Segmentation using SAM

In [3]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [4]:
sam_params = {}
sam_params['model_name'] = 'sam2'
sam_params['model_path'] = '../../semantic_SfM/sam2/sam2.1_hiera_large.pt'
sam_params['device'] = 'cuda:0'
sam_params['points_per_side'] = 32
sam_params['points_per_batch'] = 128
sam_params['pred_iou_thresh'] = 0.6
sam_params['stability_score_offset'] = 0.5
sam_params['box_nms_thresh'] = 0.6
sam_params['use_m2m'] = True


image_path_list = [os.path.join(photos_folder_path, image) for image in os.listdir(photos_folder_path)]

# sort images based on the values of keyimages in file names
image_path_list = sorted(image_path_list, key=lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))

image_list = [image for image in os.listdir(photos_folder_path)]

# sort images based on the values of keyimages in file names
image_list = sorted(image_list, key=lambda x: int(x.split('/')[-1].split('.')[0].split('_')[-1]))

In [5]:
run_segmentation = False

if run_segmentation:
    image_segmentor = ImageSegmentation(sam_params)   
    image_segmentor.set_distortion_correction(camera_path)
    image_segmentor.batch_predict(image_path_list, segmentations_folder_path, save_overlap=True, skip_existing=False)

### Simple filter

In [32]:
from ssfm.simple_mask_filter import *


In [ ]:
apply_simple_filter = False

if apply_simple_filter:
    configs = {
        'window_size': 5,
        'depth_folder': "../../data/granite_dells/associations/depth",
        'output_folder': "../../data/granite_dells/segmentations_filtered",
        'area_upper_threshold': 10,
        'area_lower_threshold': 0.001,
        'erosion_kernel_size': 5,
        'erosion_iteration':1,
        'camera_parameter_file': "../../data/granite_dells/SfM_products/cameras.xml",
        'background_mask': True
    }

    mask_filter = SimpleMaskFilter(configs)

    segmentation_folder_path = "../../data/granite_dells/segmentations"

    mask_filter.filter_batch_processes(segmentation_folder_path)

Total number of files: 316


100%|██████████| 316/316 [05:12<00:00,  1.01it/s]


In [6]:
segmentations_folder_path = "../../data/granite_dells/segmentations_filtered"

## 3. Create projection associations

In [7]:
from ssfm.probabilistic_projection import *
import time

In [10]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.01, effective_depth = np.inf)
#pointcloud_projector = PointcloudProjection()

In [11]:
pointcloud_projector.read_camera_parameters(camera_path)
pointcloud_projector.read_mesh(mesh_path)
pointcloud_projector.read_pointcloud(pointcloud_path)

In [12]:
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=8, save_depth=True)

Processing frames: 100%|██████████| 316/316 [18:14<00:00,  3.46s/it]


In [13]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [14]:
smc_solver = KeyimageAssociationsBuilder(image_list, associations_folder_path, segmentations_folder_path)

In [15]:
smc_solver.build_associations()
smc_solver.build_graph(10)

100%|██████████| 316/316 [00:51<00:00,  6.11it/s]


Building edges on GPU with 10 chunks took 14.347962379455566 seconds.


In [16]:
smc_solver.add_camera_to_graph([camera_path], camera_type="Agisoft")

../../data/granite_dells/associations/graph_with_cameras.graphml


In [17]:
smc_solver.find_min_cover()
#smc_solver.refine(0.5)

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 1772134    | 14.22                |
| Number of points covered by less than or equal to 1 image    | 2521318    | 20.24                |
| Number of points covered by less than or equal to 3 images   | 3920823    | 31.47                |
| Number of points covered by less than or equal to 5 images   | 5037165    | 40.43                |


## 4. Estimate memory usage

In [18]:
from ssfm.memory_calculator import memory_calculator

In [19]:
# pointcloud file
las_file = pointcloud_path
# image file sample; this needs to be an original image even if patch images are used
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       | 0.022966861724853516 |
| Pixel2point association for each image | 0.04593372344970703  |
| Point2pixel association for each image | 0.04641452431678772  |
|                                        |                      |
|      Segmentation for all images       |  7.257528305053711   |
| Pixel2point association for all images |  14.515056610107422  |
| Point2pixel association for all images |  14.66698968410492   |
|          pc_segmentation_ids           |  0.2320726215839386  |
|         pc_segmentation_probs          |  0.2320726215839386  |
|          keyimage_association          |   3.66674742102623   |
|                 Total                  |  40.57046726346016   |
+----------------------------------------+----------------------+


## 5. Run object registration

In [1]:
from ssfm.object_registration import *
from ssfm.post_processing import *
import time

In [21]:
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, image_list=image_list, using_graph=True, radius=2, decaying=1)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)

Processing images: 100%|██████████| 316/316 [3:20:40<00:00, 38.10s/it]   


In [22]:
image_id = 315
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path, remove_small_N=500, nearest_interpolation=500)
#add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path)

Before removing small semantics: 
maximum of semantics:  18219
number of unique semantics:  2534


After removing small semantics: 
number of unique semantics:  701


In [23]:
print(save_las_path)

# read las 
pc = laspy.read(save_las_path)
# get the semantics from the intensity
semantics = pc.intensity
semantics_ids = np.unique(semantics)
print('number of unique semantics: ', len(semantics_ids))
# print the number of points for each semantics
for i in semantics_ids:
    n = np.sum(semantics == i)
    if n < 100:
        print('semantics id: ', i, ' number of points: ', n)

../../data/granite_dells/associations/semantics/semantics_315.las
number of unique semantics:  700


In [24]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids(exclude_largest_semantic=False)
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  700


In [25]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.sort_semantic_ids(exclude_largest_semantic=False)
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_sorted.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  700
